In [1]:
library(sqldf)

Loading required package: gsubfn
Loading required package: proto
Loading required package: RSQLite


In [5]:
setwd("C:\\Users\\garavig\\Documents\\Haze\\HazeAnalysis\\Data")
airq25<-read.csv("PM2-5_2018_Daily.csv")
airq10<-read.csv("PM10_2018_Daily.csv")

In [8]:
colnames(airq25)<-c("id","d","pm25")
colnames(airq10)<-c("id","d","pm10")

In [13]:
airq<-sqldf("select a.id,a.d,a.pm25,b.pm10 from airq25 a, airq10 b where a.id=b.id and a.d=b.d")

In [14]:
airq

id,d,pm25,pm10
CU-S0001,28-02-2018,52.52500,102.73000
CU-S0002,15-02-2018,62.89826,123.37442
CU-S0002,16-02-2018,50.96376,96.69993
CU-S0002,17-02-2018,77.15833,143.26000
CU-S0002,18-02-2018,79.91827,141.49231
CU-S0002,19-02-2018,72.95452,133.82766
CU-S0002,20-02-2018,77.67401,141.67517
CU-S0002,21-02-2018,64.04096,114.19663
CU-S0002,22-02-2018,29.05366,50.32362
CU-S0002,23-02-2018,24.65841,41.67944


In [15]:
sensorlist<-read.csv("sensorlist.csv",encoding="UTF-8", stringsAsFactors=FALSE)
sensorlist[]     <- lapply(sensorlist, as.character)
Encoding(sensorlist$tambol)

[1] "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8"
[10] "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8"
[19] "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8"
[28] "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8"
[37] "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8"
[46] "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8"
[55] "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8"
[64] "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8"
[73] "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8"
[82] "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8"
[91] "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8" "UTF-8"

In [20]:
airq$date<-as.Date(airq$d,"%d-%m-%Y")

In [22]:
paste(airq$d,sep="-")

[1] "28-02-2018" "15-02-2018" "16-02-2018" "17-02-2018" "18-02-2018"
   [6] "19-02-2018" "20-02-2018" "21-02-2018" "22-02-2018" "23-02-2018"
  [11] "24-02-2018" "25-02-2018" "26-02-2018" "27-02-2018" "28-02-2018"
  [16] "01-03-2018" "02-03-2018" "03-03-2018" "04-03-2018" "05-03-2018"
  [21] "06-03-2018" "07-03-2018" "08-03-2018" "09-03-2018" "10-03-2018"
  [26] "11-03-2018" "12-03-2018" "13-03-2018" "14-03-2018" "15-03-2018"
  [31] "16-03-2018" "17-03-2018" "18-03-2018" "19-03-2018" "20-03-2018"
  [36] "21-03-2018" "22-03-2018" "23-03-2018" "27-03-2018" "28-03-2018"
  [41] "29-03-2018" "30-03-2018" "31-03-2018" "01-04-2018" "02-04-2018"
  [46] "03-04-2018" "04-04-2018" "05-04-2018" "06-04-2018" "07-04-2018"
  [51] "08-04-2018" "09-04-2018" "10-04-2018" "11-04-2018" "12-04-2018"
  [56] "13-04-2018" "14-04-2018" "15-04-2018" "16-04-2018" "17-04-2018"
  [61] "18-04-2018" "19-04-2018" "20-04-2018" "21-04-2018" "22-04-2018"
  [66] "23-04-2018" "24-04-2018" "25-04-2018" "26-04-2018" "28-04-2018"
  [71] "29-04-2018" "01-05-2018" "02-05-2018" "03-05-2018" "06-05-2018"
  [76] "07-05-2018" "08-05-2018" "09-05-2018" "10-05-2018" "11-05-2018"
  [81] "12-05-2018" "15-05-2018" "16-05-2018" "17-05-2018" "23-05-2018"
  [86] "24-05-2018" "25-05-2018" "26-05-2018" "27-05-2018" "28-05-2018"
  [91] "29-05-2018" "30-05-2018" "31-05-2018" "01-06-2018" "02-06-2018"
  [96] "04-06-2018" "06-06-2018" "07-06-2018" "15-02-2018" "16-02-2018"
 [101] "17-02-2018" "18-02-2018" "19-02-2018" "20-02-2018" "21-02-2018"
 [106] "22-02-2018" "23-02-2018" "24-02-2018" "25-02-2018" "26-02-2018"
 [111] "27-02-2018" "28-02-2018" "01-03-2018" "02-03-2018" "03-03-2018"
 [116] "04-03-2018" "05-03-2018" "06-03-2018" "07-03-2018" "08-03-2018"
 [121] "09-03-2018" "10-03-2018" "11-03-2018" "12-03-2018" "13-03-2018"
 [126] "14-03-2018" "15-03-2018" "16-03-2018" "17-03-2018" "18-03-2018"
 [131] "19-03-2018" "20-03-2018" "22-03-2018" "23-03-2018" "24-03-2018"
 [136] "25-03-2018" "26-03-2018" "27-03-2018" "28-03-2018" "29-03-2018"
 [141] "30-03-2018" "31-03-2018" "01-04-2018" "02-04-2018" "03-04-2018"
 [146] "04-04-2018" "05-04-2018" "06-04-2018" "07-04-2018" "08-04-2018"
 [151] "09-04-2018" "10-04-2018" "11-04-2018" "12-04-2018" "13-04-2018"
 [156] "14-04-2018" "15-04-2018" "16-04-2018" "17-04-2018" "18-04-2018"
 [161] "19-04-2018" "20-04-2018" "21-04-2018" "22-04-2018" "23-04-2018"
 [166] "24-04-2018" "25-04-2018" "26-04-2018" "27-04-2018" "28-04-2018"
 [171] "29-04-2018" "30-04-2018" "01-05-2018" "02-05-2018" "03-05-2018"
 [176] "04-05-2018" "05-05-2018" "06-05-2018" "07-05-2018" "08-05-2018"
 [181] "09-05-2018" "10-05-2018" "11-05-2018" "12-05-2018" "13-05-2018"
 [186] "14-05-2018" "15-05-2018" "20-05-2018" "21-05-2018" "22-05-2018"
 [191] "23-05-2018" "24-05-2018" "25-05-2018" "26-05-2018" "27-05-2018"
 [196] "28-05-2018" "29-05-2018" "30-05-2018" "31-05-2018" "01-06-2018"
 [201] "02-06-2018" "03-06-2018" "04-06-2018" "05-06-2018" "06-06-2018"
 [206] "07-06-2018" "15-02-2018" "16-02-2018" "17-02-2018" "18-02-2018"
 [211] "19-02-2018" "20-02-2018" "21-02-2018" "22-02-2018" "23-02-2018"
 [216] "24-02-2018" "25-02-2018" "26-02-2018" "27-02-2018" "28-02-2018"
 [221] "01-03-2018" "02-03-2018" "03-03-2018" "04-03-2018" "05-03-2018"
 [226] "06-03-2018" "07-03-2018" "08-03-2018" "09-03-2018" "10-03-2018"
 [231] "11-03-2018" "12-03-2018" "13-03-2018" "14-03-2018" "15-03-2018"
 [236] "16-03-2018" "17-03-2018" "18-03-2018" "19-03-2018" "20-03-2018"
 [241] "21-03-2018" "22-03-2018" "23-03-2018" "24-03-2018" "25-03-2018"
 [246] "26-03-2018" "27-03-2018" "28-03-2018" "29-03-2018" "30-03-2018"
 [251] "31-03-2018" "01-04-2018" "02-04-2018" "03-04-2018" "04-04-2018"
 [256] "05-04-2018" "06-04-2018" "07-04-2018" "08-04-2018" "09-04-2018"
 [261] "10-04-2018" "11-04-2018" "12-04-2018" "13-04-2018" "14-04-2018"
 [266] "15-04-2018" "16-04-2018" "17-04-2018" "18-04-2018" "19-04-2018"
 [271] "20-04-2018" "21-04-2018" "22-04-2018" "23-04-2018" "24-04-2018"
 [276] "25-04-2018" "26-04-2018" "27-04-201